In [59]:
# X &. Y
# Dummify cat variables
# scale cont. variables
# concat new 

In [60]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, precision_score, accuracy_score, recall_score, f1_score, roc_curve, auc
from sklearn.preprocessing import OneHotEncoder
from imblearn.over_sampling import SMOTE

import warnings
warnings.filterwarnings('ignore')

In [61]:
df_mod = pd.read_csv('Terry_Cleaned.csv')
df_mod = df_mod.drop(['Unnamed: 0','Officer Squad'], axis=1)
df_mod.head()

,Subject Age Group,Stop Resolution,Officer YOB,Officer Gender,Officer Race,Subject Perceived Race,Subject Perceived Gender,Weapon,Call Made,Arrest Made,Frisk,Year Reported
0,26 - 35,Arrest,1984,M,Black or African American,Asian,Male,0,0,0,0,2015
1,26 - 35,Field Contact,1965,M,White,Other,Male,0,0,0,0,2015
2,26 - 35,Field Contact,1961,M,White,White,Male,0,0,0,0,2015
3,26 - 35,Field Contact,1963,M,White,Other,Male,0,0,0,0,2015
4,26 - 35,Field Contact,1977,M,White,Black or African American,Male,0,0,0,0,2015


In [62]:
df_mod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43063 entries, 0 to 43062
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Subject Age Group         43063 non-null  object
 1   Stop Resolution           43063 non-null  object
 2   Officer YOB               43063 non-null  int64 
 3   Officer Gender            43063 non-null  object
 4   Officer Race              43063 non-null  object
 5   Subject Perceived Race    43063 non-null  object
 6   Subject Perceived Gender  43063 non-null  object
 7   Weapon                    43063 non-null  int64 
 8   Call Made                 43063 non-null  int64 
 9   Arrest Made               43063 non-null  int64 
 10  Frisk                     43063 non-null  int64 
 11  Year Reported             43063 non-null  int64 
dtypes: int64(6), object(6)
memory usage: 3.9+ MB


In [63]:
# Check for class imbalance
print(df_mod['Arrest Made'].value_counts())

0    40890
1     2173
Name: Arrest Made, dtype: int64


In [64]:
# Create X and y
y = df_mod['Arrest Made']
X = df_mod.drop(['Arrest Made'], axis=1)

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Remove object-type features from X
cont_features = X.select_dtypes(exclude = ['object'])

# Remove object-type features X_train and X_test
X_train_cont = X_train.select_dtypes(exclude = ['object'])
X_test_cont = X_test.select_dtypes(exclude = ['object'])


In [65]:
# Scale the train and test data
ss = StandardScaler()
X_train_ss = ss.fit_transform(X_train_cont)
X_test_ss = ss.transform(X_test_cont)

In [66]:
# Create a dataframe only containing the object-type features

X_train_cat = X_train.select_dtypes(include = ['object'])
X_test_cat = X_test.select_dtypes(include = ['object'])

In [67]:
# OneHotEncode categorical variables
ohe = OneHotEncoder(handle_unknown='ignore')

# Transform training and test sets
X_train_ohe = ohe.fit_transform(X_train_cat)
X_test_ohe = ohe.transform(X_test_cat)

# Convert these columns into a DataFrame
columns = ohe.get_feature_names(input_features=X_train_cat.columns) # Build an array of column names
cat_train_df = pd.DataFrame(X_train_ohe.todense(), columns=columns) # Return a dense version of the dataframe
cat_test_df = pd.DataFrame(X_test_ohe.todense(), columns=columns)

In [68]:
# Combine scaled training data and categorcial data
X_train_all = pd.concat([pd.DataFrame(X_train_ss), cat_train_df])
X_test_all = pd.concat([pd.DataFrame(X_test_ss), cat_test_df])

In [69]:
# Check for class imbalance
print(y_train.value_counts())

# Fit SMOTE to training data
smote = SMOTE()
X_train_resampled, y_train_resampled = smote.fit_sample(X_train_all, y_train)


0    30634
1     1663
Name: Arrest Made, dtype: int64


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
# Separate continuous and categorical columns
# continuous = ['Officer YOB', 'Weapon', 'Call Made', 'Arrest Made', 'Frisk', 'Year Reported']
# categorical = ['Subject Age Group', 'Stop Resolution', 'Officer Gender', 'Officer Race', 'Subject Perceived Race', 'Subject Perceived Gender']

In [ ]:
# # Dummify categorical variables
# dummies = pd.get_dummies(df_mod[categorical])
# print(dummies.shape)
# dummies